In [ ]:
####Topic sharing Duration time estimation####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

In [ ]:
#多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

#切断ポアソン分布の乱数を生成する関数
def rtpois(mu, a, b, n):
    if a==0:
        FA = np.exp(-mu)
    else:
        FA = scipy.stats.poisson.cdf(a, mu)
    if b==np.inf:
        FB = np.repeat(1.0, n)
    else:
        FB = scipy.stats.poisson.cdf(b, mu)
    return np.array(scipy.stats.poisson.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu), dtype="int")

#ポアソン分布の対数尤度
def loglike(mu, y, y_factorial):
    LL = y * np.log(mu) - mu - y_factorial
    return LL

In [ ]:
####データの生成####
##データの定義
k1 = 5
k2 = 6
k_vec1 = np.repeat(1.0, k1)
k_vec2 = np.repeat(1.0, k2)
place = 30
hh = 30000
pt = rtpois(0.5, 0, np.inf, hh)
hhpt = np.sum(pt)
d_id = np.repeat(np.arange(hh), pt)
d_list = [i for i in range(hh)]
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")

In [ ]:
##訪問店舗を生成
#パラメータの定義
topic = 20
theta0 = np.random.dirichlet(np.repeat(5.0, topic), 1).reshape(-1)
phi0 = np.random.dirichlet(np.repeat(0.1, place), topic)
z0 = np.dot(np.random.multinomial(1, theta0, hh), np.arange(topic))[d_id, ]

#多項分布で店舗を選択
place_id1 = np.repeat(0, hhpt)
for i in range(hh):
    index = d_list[i]
    if pt[i]==1:
        place_id1[index] = np.dot(np.random.multinomial(1, phi0[z0[index], ].reshape(-1), pt[i]), np.arange(place))
    else:
        while True:
            get_place = rmnom(phi0[z0[index], ], pt[i], place, 1)[1]
            if np.max(np.sum(get_place, axis=0))==1:
                place_id1[index] = np.dot(get_place, np.arange(place))
                break
                
#インデックスを定義
place_list1 = [i for i in range(place)]
place_vec1 = [i for i in range(place)]
place_n1 = np.repeat(0, place)
for i in range(place):
    place_list1[i] = np.array(np.where(place_id1==i)[0], dtype="int")
    place_vec1[i] = np.repeat(1.0, place_list1[i].shape[0])
    place_n1[i] = place_list1[i].shape[0]

In [ ]:
##訪問頻度を生成
#パーセンタイルを設定
normalized = (np.arange(k1) - np.mean(np.arange(k1))) / np.std(np.arange(k1)) 
percentile_value1 = 20*(np.exp((2.25*normalized)) / (1 + np.exp((2.25*normalized))))[::-1]
percentile_value2 = 100*np.exp(3*normalized) / (1 + np.exp(3*normalized))

#パラメータの生成
theta1 = np.random.dirichlet(percentile_value1, place)
beta1  = np.abs(np.random.normal(1.0, 0.2, k1*place)).reshape(place, k1)
gamma = np.percentile(np.random.gamma(1.5, 20, 10000), percentile_value2)
thetat1 = theta1.copy()
betat1 = beta1.copy(); gammat = gamma.copy()

#トピックと選択店舗から訪問頻度を生成
z1, Z1 = rmnom(theta1[place_id1, ], hhpt, k1, 1)
mu1 = np.dot(Z1*beta1[place_id1, ], k_vec1) * gamma[z1]
freq = rtpois(mu1, 0, np.inf, hhpt)

In [ ]:
#訪問回数を可視化
colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
fig = plt.figure(figsize=(12.5, 6.0))
ax = fig.add_subplot(2, 3, 1)
plt.hist(freq, bins=25, range=(0, np.max(freq)))
plt.title("訪問回数の分布", fontsize=12.5)
for j in range(k1):
    ax = fig.add_subplot(2, 3, j+2)
    ax.hist(freq[z1==j],  bins=25, range=(0, np.max(freq)), color=colorlist[j])
    plt.title("トピック" + str(j) + "の訪問回数の分布", fontsize=12.5)
fig.tight_layout()
plt.show()

In [ ]:
##訪問頻度の観測データを設定
#IDを定義
n = d_id.shape[0]
N = np.sum(freq)
exit_id = np.repeat(d_id, freq)
place_id2 = np.repeat(place_id1, freq)

#ユーザーインデックスを定義
exit_list = [i for i in range(hh)]
exit_vec = [i for i in range(hh)]
exit_n = [i for i in range(hh)]
for i in range(hh):
    exit_list[i] = np.array(np.where(exit_id==i)[0], dtype="int")
    exit_vec[i] = np.repeat(1.0, exit_list[i].shape[0])
    exit_n[i] = exit_list[i].shape[0]

#店舗インデックスを定義
place_list2 = [i for i in range(place)]
place_vec2 = [i for i in range(place)]
place_n2 = np.repeat(0, place)
for i in range(place):
    place_list2[i] = np.array(np.where(place_id2==i)[0], dtype="int")
    place_vec2[i] = np.repeat(1.0, place_list2[i].shape[0])
    place_n2[i] = place_list2[i].shape[0]

In [ ]:
##滞在時間を生成
#パラメータを生成
#パーセンタイルを設定
normalized = (np.arange(k2) - np.mean(np.arange(k2))) / np.std(np.arange(k2)) 
percentile_value1 = (np.exp((1.25*normalized)) / (1 + np.exp((1.25*normalized))))[::-1]
percentile_value2 = 100*np.exp(2*normalized) / (1 + np.exp(2*normalized))
percentile_value3 = 100*np.exp(2.75*normalized) / (1 + np.exp(2.75*normalized))
percentile_value3[0:2] = np.array([5.0, 12.5])
percentile_value2[k2-1] = 99.5; percentile_value3[k2-1] = 99.0

#パラメータを生成
theta2 = np.random.dirichlet(percentile_value1, hh)
beta2 = np.abs(np.random.normal(1.0, 0.1, k2*place)).reshape(place, k2)
beta3 = np.abs(np.random.normal(1.0, 0.1, k2*place)).reshape(place, k2)
kappa = np.percentile(np.random.gamma(1.5, 10, 10000), percentile_value2)
lkappa = np.log(kappa)
phi = np.percentile(np.random.gamma(0.75, 125, 10000), percentile_value3) / kappa
thetat2 = theta2.copy()
betat2 = beta2.copy(); betat3 = beta3.copy()
kappat = kappa.copy(); phit = phi.copy()

#トピックから滞在時間を生成
z2, Z2 = rmnom(theta2[exit_id, ], N, k2, 1)
mu2 = np.dot(Z2 * beta2[place_id2, ], k_vec2) * phi[z2, ]
Lambda = np.dot(Z2 * beta3[place_id2, ], k_vec2) * kappa[z2, ]
exit_time = np.random.gamma(Lambda, mu2, N)
y_log = np.log(exit_time)

In [ ]:
#滞在時間を可視化
colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
fig = plt.figure(figsize=(12.5, 9.0))
ax = fig.add_subplot(3, 3, 1)
plt.hist(exit_time, bins=25, range=(0, np.max(exit_time)))
plt.title("滞在時間の分布", fontsize=12.5)
for j in range(k2):
    ax = fig.add_subplot(3, 3, j+2)
    ax.hist(exit_time[z2==j],  bins=25, range=(0, np.max(exit_time)), color=colorlist[j])
    plt.title("トピック" + str(j) + "の滞在時間の分布", fontsize=12.5)
fig.tight_layout()
plt.show()

In [ ]:
####Topic sharing Duration time estimation modelを推定####
##ガンマ分布のパラメータを推定するための関数
#ガンマ分布の確率密度関数
def Gamma_pdf(kappa, mu_vec, y_vec1):
    inv_Lambda = 1 / mu_vec
    Lho = (np.power(inv_Lambda, kappa) / scipy.special.gamma(kappa)) * np.power(y_vec1, kappa-1) * np.exp(-inv_Lambda*y_vec1)
    return Lho

##共通の形状パラメータをサンプリングするための関数
def Lho_kappa(lkappa, delta, tau, beta, mu, exit_time, y_log, index_z, place_id, k):
    #パラメータを定義
    kappa = np.exp(lkappad)
    inv_Lambda = 1 / mu
    lambda_y = inv_Lambda*exit_time
    lgamma = scipy.special.loggamma(beta*kappa)[place_id, ]
    new_kappa = (beta*kappa)[place_id, ]

    #対数事後分布の和
    Posterior = np.repeat(0.0, k)
    for j in range(k):
        index = index_z[j]
        Lho = np.sum(new_kappa[index, j]*np.log(inv_Lambda[index]) - lgamma[index, j] + 
                        (new_kappa[index, j]-1)*y_log[index] - lambda_y[index])
        Prior = -1/2 * np.power(lkappa[j] - delta, 2) / tau
        Posterior[j] = Lho + Prior    
    return Posterior

#共通の形状パラメータの勾配ベクトル
def dll_kappa(lkappa, delta, tau, beta, mu, exit_time, place_id, index_z, k):
    #パラメータを定義
    kappa = np.exp(lkappa)
    inv_Lambda = 1 / mu
    digamma = scipy.special.digamma(kappa)

    #形状パラメータの勾配ベクトル
    gradient = np.repeat(0.0, k)
    for j in range(k):
        index = index_z[j]
        dll = np.sum(beta[place_id[index], j]*np.log(inv_Lambda[index]) - digamma[j] + np.log(exit_time[index]))
        dmv = -(lkappa[j] - delta) / tau
        gradient[j] = -(dll + dmv)
    return gradient

#共通の形状パラメータの数値微分
def diff_kappa(lkappa, delta, tau, beta, mu, exit_time, y_log, place_id, index_z, k):
    #パラメータを定義
    h = 1e-10
    kappa1 = np.exp(lkappa + h)
    kappa2 = np.exp(lkappa - h)
    lgamma1 = scipy.special.loggamma(beta*(kappa1))[place_id, ]
    lgamma2 = scipy.special.loggamma(beta*(kappa2))[place_id, ]
    new_kappa1 = (beta*kappa1)[place_id, ]
    new_kappa2 = (beta*kappa2)[place_id, ]
    inv_Lambda = 1 / mu
    lambda_y = inv_Lambda*exit_time

    #勾配ベクトルの和を定義
    gradient = np.repeat(0.0, k)
    for j in range(k):
        index = index_z[j]
        inv_lambda = inv_Lambda[index]
        lambda_log = np.log(inv_lambda)
        Lho1 = np.sum(new_kappa1[index, j]*lambda_log - lgamma1[index, j] + (new_kappa1[index, j]-1)*y_log[index] - lambda_y[index])
        Lho2 = np.sum(new_kappa2[index, j]*lambda_log - lgamma2[index, j] + (new_kappa2[index, j]-1)*y_log[index] - lambda_y[index])
        dll = (Lho1 - Lho2) / (2*h)
        dmv = -(lkappa[j] - delta) / tau
        gradient[j] = -(dll + dmv)
    return gradient

#共通のリープフロッグ法を解く関数
def leapfrog_kappa(r, z1, D, e, L): 
    def leapfrog_step(r, z1, e):
        r2 = r - e * D(z1, delta1, tau1, beta3, mu, exit_time, y_log, place_id2, index_z2, k2) / 2
        z2 = z1 + e * r2
        r2 = r2 - e * D(z2, delta1, tau1, beta3, mu, exit_time, y_log, place_id2, index_z2, k2) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z1]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result


##店舗ごとの形状パラメータをサンプリングするための関数
#店舗ごとの対数事後分布の和
def Lho_beta(lbeta, delta, tau, kappa, Z, exit_time0, y_log0, inv_Lambda, lambda_log, place, place_id, place_list, k):
    #パラメータを定義
    beta = np.exp(lbeta)
    lgamma = Z * scipy.special.loggamma(beta*kappa)[place_id, ]
    new_kappa = Z * (beta*kappa)[place_id, ]
    lambda_y = inv_Lambda*exit_time0

    #対数事後分布の和
    Posterior = np.zeros((place, k))
    for i in range(place):
        index = place_list[i]
        Lho = np.sum(new_kappa[index, ]*lambda_log[index, ] - lgamma[index, ] +
                     (new_kappa[index, ]-Z[index, ])*y_log0[index, ] - lambda_y[index, ], axis=0)
        Prior = -1/2 * np.power(lbeta[i, ] - delta, 2) / tau
        Posterior[i, ] = Lho + Prior
    return Posterior 

#店舗ごとの形状パラメータの数値微分
def diff_beta(lbeta, delta, tau, kappa, Z, exit_time0, y_log0, inv_Lambda, lambda_log, place, place_id, place_list, k):
    #パラメータを定義
    h = 1e-10
    beta_h1 = np.exp(lbeta + h)
    beta_h2 = np.exp(lbeta - h)
    lgamma1 = Z * scipy.special.loggamma(beta_h1*kappa)[place_id, ]
    lgamma2 = Z * scipy.special.loggamma(beta_h2*kappa)[place_id, ]
    new_kappa1 = Z * (beta_h1*kappa)[place_id, ]
    new_kappa2 = Z * (beta_h2*kappa)[place_id, ]
    lambda_y = inv_Lambda*exit_time0

    #勾配ベクトルの和を定義
    gradient = np.zeros((place, k))
    for i in range(place):
        index = place_list[i]
        Lho1 = np.sum(new_kappa1[index, ]*lambda_log[index, ] - lgamma1[index, ] +
                      (new_kappa1[index, ]-Z2[index, ])*y_log0[index, ] - lambda_y[index, ], axis=0)
        Lho2 = np.sum(new_kappa2[index, ]*lambda_log[index, ] - lgamma2[index, ] + 
                      (new_kappa2[index, ]-Z2[index, ])*y_log0[index, ] - lambda_y[index, ], axis=0)
        dll = (Lho1 - Lho2) / (2*h)
        dmv = -(lbeta[i, ] - delta) / tau
        gradient[i, ] = -(dll + dmv)
    return gradient

#店舗ごとのリープフロッグ法を解く関数
def leapfrog_beta(r, z1, D, e, L): 
    def leapfrog_step(r, z1, e):
        r2 = r - e * D(z1, delta2, tau2, kappa, Zi2, exit_time0, y_log0, inv_Lambda, lambda_log, place, place_id2, place_list2, k2) / 2
        z2 = z1 + e * r2
        r2 = r2 - e * D(z2, delta2, tau2, kappa, Zi2, exit_time0, y_log0, inv_Lambda, lambda_log, place, place_id2, place_list2, k2) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z1]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [ ]:
##アルゴリズムの設定
#MCMCの設定
R = 2000
keep = 2
burnin = 500
iter = 0
disp = 10
e1 = 0.001
e2 = 0.0025
L1 = 3
L2 = 2

#事前分布の設定
alpha1 = 0.25
alpha2 = 0.1
delta1 = 0.0
delta2 = 1.0
tau1 = 1.0
tau2 = 0.1
s01 = 0.1
v01 = 0.1
s02 = 0.25
v02 = 0.25

In [ ]:
##パラメータの真値
#トピック分布の真値
theta1 = thetat1.copy()
theta2 = thetat2.copy()

#モデルパラメータの真値
beta1 = betat1.copy()
beta2 = betat2.copy()
beta3 = betat3.copy()
lbeta = np.log(beta3)
gamma = gammat.copy()
phi = phit.copy()
kappa = kappat.copy()
lkappa = np.log(kappa)

#トピックの真値
Zi1 = Z1.copy()
Zi2 = Z2.copy()
z1 = np.dot(Zi1, np.arange(k1))
z2 = np.dot(Zi2, np.arange(k2))
index_z1 = [np.array(np.where(Zi1[:, j]==1)[0], dtype="int") for j in range(k1)]
index_z2 = [np.array(np.where(Zi2[:, j]==1)[0], dtype="int") for j in range(k2)]

#期待値の真値
mut1 = np.dot(Zi1 * beta1[place_id1, ], k_vec1) * gamma[z1]
mut2 = np.dot(Zi2 * beta2[place_id2, ], k_vec2) * phi[z2]
Lambdat = np.dot(Zi2 * beta3[place_id2, ], k_vec2) * kappa[z2, ]

In [ ]:
##パラメータの初期値
#分位点を定義
normalized1 = (np.arange(k1) - np.mean(np.arange(k1))) / np.std(np.arange(k1)) 
normalized2 = (np.arange(k2) - np.mean(np.arange(k2))) / np.std(np.arange(k2)) 
percentile_value1 = 100 * (np.exp((2.75*normalized1)) / (1 + np.exp(2.75*normalized1)))
percentile_value2 = 100 * (np.exp(2.75*normalized2) / (1 + np.exp(2.75*normalized2)))

#トピック分布の初期値
theta1 = np.random.dirichlet(np.repeat(1.0, k1), place)
theta2 = np.random.dirichlet(np.repeat(1.0, k2), hh)

#モデルパラメータの初期値
beta1 = np.abs(np.random.normal(1.0, 0.01, k1*place).reshape(place, k1))
beta2 = np.abs(np.random.normal(1.0, 0.01, k2*place).reshape(place, k2))
beta3 = np.abs(np.random.normal(1.0, 0.01, k2*place).reshape(place, k2))
gamma = np.percentile(freq, percentile_value1)
lkappa = np.repeat(3.0, k2)
kappa = np.exp(lkappa)
phi = np.percentile(exit_time, percentile_value2) / kappa

#トピックの初期値
z1, Zi2 = rmnom(theta1[place_id1, ], hhpt, k1, 1)
z2, Zi2 = rmnom(theta2[exit_id, ], N, k2, 1)
index_z1 = [np.array(np.where(Zi1[:, j]==1)[0], dtype="int") for j in range(k1)]
index_z2 = [np.array(np.where(Zi2[:, j]==1)[0], dtype="int") for j in range(k2)]

#期待値の初期値
mu1 = np.dot(Zi1 * beta1[place_id1, ], k_vec1) * gamma[z1]
mu2 = np.dot(Zi2 * beta2[place_id2, ], k_vec2) * phi[z2]
Lambda = np.dot(Zi2 * beta3[place_id2, ], k_vec2) * kappa[z2, ]

In [ ]:
##パラメータの格納用配列
#モデルパラメータの格納用配列
THETA1 = np.zeros((place, k1, int(R/keep)))
THETA2 = np.zeros((hh, k2, int(R/keep)))
BETA1 = np.zeros((place, k1, int(R/keep)))
BETA2 = np.zeros((place, k2, int(R/keep)))
BETA3 = np.zeros((place, k2, int(R/keep)))
GAMMA = np.zeros((int(R/keep), k1))
PHI = np.zeros((int(R/keep), k2))
KAPPA = np.zeros((int(R/keep), k2))

#トピックの格納用配列
SEG1 = np.zeros((int(R/keep), hhpt), dtype="int8")
SEG2 = np.zeros((int(R/keep), N), dtype="int8")

In [ ]:
##対数尤度の基準値
#1パラメータモデルの対数尤度
LLst1 = np.sum(scipy.stats.poisson.logpmf(freq, np.mean(freq)))
LLst2 = np.sum(scipy.stats.gamma.logpdf(exit_time, 1.0, scale=np.mean(exit_time)))
print(np.round([LLst1, LLst2], 1))

#真値での対数尤度
LLbest1 = np.sum(scipy.stats.poisson.logpmf(freq, mut1))
LLbest2 = np.sum(scipy.stats.gamma.logpdf(exit_time, Lambdat, scale=mut2))
print(np.round([LLbest1, LLbest2], 1))

In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##トピックをサンプリング
    #モデルの事後分布を定義
    mu_deploy1 = beta1[place_id1, ] * gamma
    mu_deploy2 = beta2[place_id2, ] * phi
    lambda_deploy = beta3[place_id2, ] * kappa
    Posterior1 = theta1[place_id1, ] * scipy.stats.poisson.pmf(freq[:, np.newaxis], mu_deploy1)
    Posterior2 = thetat2[exit_id, ] * Gamma_pdf(lambda_deploy, mu_deploy2, exit_time[:, np.newaxis])

    #潜在変数の割当確率からトピックをサンプリング
    Prob1 = Posterior1 / np.dot(Posterior1, k_vec1)[:, np.newaxis]
    Prob2 = Posterior2 / np.dot(Posterior2, k_vec2)[:, np.newaxis]
    z1, Zi1 = rmnom(Prob1, hhpt, k1, 1)
    z2, Zi2 = rmnom(Prob2, N, k2, 1)

    #トピックのインデックスを定義
    index_z1 = [np.array(np.where(Zi1[:, j]==1)[0], dtype="int") for j in range(k1)]
    index_z2 = [np.array(np.where(Zi2[:, j]==1)[0], dtype="int") for j in range(k2)]


    ##トピック分布をサンプリング
    #訪問頻度のトピック分布をサンプリング
    for j in range(place):
        wsum = np.dot(Zi1[place_list1[j], ].T, place_vec1[j]) + alpha1
        theta1[j, ] = np.random.dirichlet(wsum, 1)

    #滞在時間のトピック分布をサンプリング
    for i in range(hh):
        wsum = np.dot(Zi2[exit_list[i], ].T, exit_vec[i]) + alpha2
        theta2[i, ] = np.random.dirichlet(wsum, 1)


    ##訪問頻度のパラメータをサンプリング
    #共通のパラメータをサンプリング
    s = np.sum(Zi1 * freq[:, np.newaxis], axis=0) + s01
    v = np.sum(Zi1 * beta1[place_id1, ], axis=0) + v01
    gamma = np.random.gamma(s, 1/v, k1)

    #店舗ごとのパラメータをサンプリング
    s = np.zeros((place, k1))
    v = np.zeros((place, k1))
    for j in range(place):
        index = place_list1[j]
        s[j, ] = np.sum(Zi1[index, ] * freq[index, np.newaxis], axis=0) + s01
        v[j, ] = np.sum(Zi1[index, ] * gamma, axis=0) + v01
    beta1 = np.random.gamma(s, 1/v)


    ##滞在時間の尺度パラメータをサンプリング
    #共通のパラメータをサンプリング
    s = np.sum(Zi2 * exit_time[:, np.newaxis], axis=0) + s02
    v = np.sum(Zi2 * beta2[place_id2, ] * beta3[place_id2, ] * kappa, axis=0) + v02
    phi = np.random.gamma(s, 1/v, k2)

    #店舗ごとのパラメータのサンプリング
    s = np.zeros((place, k2))
    v = np.zeros((place, k2))
    for j in range(place):
        index = place_list2[j]
        s[j, ] = np.sum(Zi2[index, ] * exit_time[index, np.newaxis], axis=0) + s02
        v[j, ] = np.sum(Zi2[index, ] * beta3[place_id2[index], ] * kappa * phi, axis=0) + v02
    beta2 = np.random.gamma(s, 1/v)


    ##HMCで滞在時間の共通の形状パラメータをサンプリング
    #リープフロッグ法で新しいパラメータをサンプリング
    mu = np.dot(Zi2 * beta2[place_id2, ], k_vec2) * phi[z2]
    rold = np.random.normal(0, 1.0, k2)
    lkappad = lkappa.copy()
    res = leapfrog_kappa(rold, lkappad, diff_kappa, e1, L1)
    rnew = res[0]
    lkappan = res[1]

    #移動前と移動後のハミルトニアン
    Hnew = -Lho_kappa(lkappan, delta1, tau1, beta3, mu, exit_time, y_log, index_z2, place_id2, k2)
    Hold = -Lho_kappa(lkappad, delta1, tau1, beta3, mu, exit_time, y_log, index_z2, place_id2, k2)

    #新しいパラメータを採択
    rand = np.random.uniform(0, 1, k2)
    psi = np.exp(Hold - Hnew); psi[psi > 1.0] = 1.0
    accept_prob1 = psi.copy()
    flag = np.array(psi > rand, dtype="int")
    lkappa = flag*lkappan + (1-flag)*lkappad
    kappa = np.exp(lkappa)


    ##HMCで店舗ごとの形状パラメータをサンプリング
    #新しい変数を定義
    exit_time0 = Zi2 * exit_time[:, np.newaxis]
    y_log0 = Zi2 * y_log[:, np.newaxis]
    mu_deploy = beta2[place_id2, ] * phi
    inv_Lambda = Zi2 * (1 / mu_deploy) 
    lambda_log = Zi2 * np.log(1 / mu_deploy)

    #リープフロッグ法で新しいパラメータをサンプリング
    rold = np.random.normal(0, 1.0, k2*place).reshape(place, k2)
    lbetad = lbeta.copy()
    res = leapfrog_beta(rold, lbetad, diff_beta, e2, L2)
    rnew = res[0]
    lbetan = res[1]

    #移動前と移動後のハミルトニアン
    Lho_new = Lho_beta(lbetan, delta2, tau2, kappa, Zi2, exit_time0, y_log0, inv_Lambda, lambda_log, place, place_id2, place_list2, k2)
    Lho_old = Lho_beta(lbetad, delta2, tau2, kappa, Zi2, exit_time0, y_log0, inv_Lambda, lambda_log, place, place_id2, place_list2, k2)
    Hnew = -Lho_new + np.power(rnew, 2)/2
    Hold = -Lho_old + np.power(rold, 2)/2

    #新しいパラメータを採択
    rand = np.random.uniform(0, 1, k2*place).reshape(place, k2)
    psi = np.exp(Hold - Hnew); psi[psi > 1.0] = 1.0
    accept_prob2 = psi.copy()
    flag = np.array(psi > rand, dtype="int")
    lbeta = flag*lbetan + (1-flag)*lbetad
    beta3 = np.exp(lbeta)


    ##サンプリング結果の格納と表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = int(rp/keep)
        THETA1[:, :, mkeep] = theta1
        THETA2[:, :, mkeep] = theta2
        BETA1[:, :, mkeep] = beta1
        BETA2[:, :, mkeep] = beta2
        BETA3[:, :, mkeep] = beta3
        GAMMA[mkeep, ] = gamma
        PHI[mkeep, ] = phi
        KAPPA[mkeep, ] = kappa
        SEG1[mkeep, ] = z1
        SEG2[mkeep, ] = z2

    #対数尤度の更新とサンプリング結果の表示
    if rp%disp==0:  
        #期待値の更新
        mu1 = np.dot(Zi1 * beta1[place_id1, ], k_vec1) * gamma[z1]
        mu2 = np.dot(Zi2 * beta2[place_id2, ], k_vec2) * phi[z2]
        Lambda = np.dot(Zi2 * beta3[place_id2, ], k_vec2) * kappa[z2, ]

        #対数尤度の更新
        LL1 = np.sum(scipy.stats.poisson.logpmf(freq, mu1))
        LL2 = np.sum(scipy.stats.gamma.logpdf(exit_time, Lambda, scale=mu2))

        #サンプリング結果の表示
        print(rp)
        print(np.round(accept_prob1, 3))
        print(np.round(np.mean(accept_prob2, axis=0), 3))
        print(np.round(np.vstack((gamma, gammat)), 3))
        print(np.round(np.vstack((phi*kappa, phit*kappat)), 3))
        print(np.round(np.vstack((kappa, kappat)), 3))
        print(np.round([LL1, LLst1, LLbest1], 1))
        print(np.round([LL2, LLst2, LLbest2], 1))